In [18]:
import pandas

In [19]:
dataset_dir = "datasets"
filename_22k = f'{dataset_dir}\\df_vacancies_full_ru_22K.csv'
filename_4k = f'{dataset_dir}\\hh_corpus_4K_classter.csv'

df_vacancies_22k = pandas.read_csv(filename_22k, sep = ',', encoding='utf')
df_vacancies_4k = pandas.read_csv(filename_4k, sep = ',', encoding='utf')

In [20]:
del df_vacancies_22k['lemmatized_text_pos_tags']

In [21]:
df_vacancies_22k.head()

,id,is_prog,is_test,label_true,title,text
0,24992389,False,True,101,Специалист офиса продаж и обслуживания,Специалист офиса продаж и обслуживания Ты наш ...
1,25195139,False,True,101,Специалист салона связи Beeline,Специалист салона связи Beeline Ты наш человек...
2,22995930,False,True,101,Специалист отдела продаж Билайн,Специалист отдела продаж Билайн Ты наш человек...
3,23754493,False,True,101,Продавец-консультант,"Продавец-консультант Ты наш человек, если поня..."
4,23841958,False,True,101,Продавец консультант в салон связи Билайн,Продавец консультант в салон связи Билайн Ты н...


In [11]:
df_vacancies_4k.head()

,id,title,text,labels
0,20001586,Инженер-проектировщик решений по передаче данных,Инженер-проектировщик решений по передаче данн...,20
1,20001746,Архитектор решений,Архитектор решений Обязанности: разработка ар...,17
2,20001909,Python-разработчик в Район,Python-разработчик в Район Новый сервис Яндекс...,14
3,20002793,Системный администратор/Программист Delphi,Системный администратор/Программист Delphi Обя...,"14,10"
4,20004542,IT аналитик (электронные торговые площадки),IT аналитик (электронные торговые площадки) ...,8


# Готовим тексты вакансий

In [13]:
import codecs
import os
import pymorphy2
from string import ascii_lowercase, digits, whitespace
from tqdm import tqdm_notebook

morph = pymorphy2.MorphAnalyzer()

cyrillic = u"абвгдеёжзийклмнопрстуфхцчшщъыьэюя"

allowed_characters = ascii_lowercase + digits + cyrillic + whitespace

def complex_preprocess(text, additional_allowed_characters = "+#"):
    return ''.join([character if character in set(allowed_characters+additional_allowed_characters) else ' ' for character in text.lower()]).split()


def lemmatize(tokens, filter_pos):
    '''Produce normal forms for russian words using pymorphy2
    '''
    lemmas = []
    tagged_lemmas = []
    
    for token in tokens:
        parsed_token = morph.parse(token)[0]
        norm = parsed_token.normal_form
        pos = parsed_token.tag.POS
        if pos is not None:
            if pos not in filter_pos:
                lemmas.append(norm)
                tagged_lemmas.append(norm + "_" + pos)
        else:
            lemmas.append(token)
            tagged_lemmas.append(token+"_")

    return lemmas, tagged_lemmas

def process_text(full_text, filter_pos=("PREP", "NPRO", "CONJ")):
    '''Process a single text and return a processed version
    '''
    single_line_text = full_text.replace('\n',' ')
    preprocessed_text = complex_preprocess(single_line_text)
    
    lemmatized_text, lemmatized_text_pos_tags = lemmatize(preprocessed_text, filter_pos=filter_pos)

    return { "full_text" : full_text,
    "single_line_text": single_line_text,
    "preprocessed_text": preprocessed_text,
    "lemmatized_text": lemmatized_text,
    "lemmatized_text_pos_tags": lemmatized_text_pos_tags}

def process_text_documents(text_files_directory, filter_pos=("PREP", "NPRO", "CONJ")):
    for file in os.listdir(text_files_directory):
        if os.path.isfile(os.path.join(text_files_directory, file)):
            with codecs.open(os.path.join(text_files_directory, file), encoding='utf-8') as f:
                full_text = f.read()
                doc_dict = process_text(full_text)
                doc_dict["filename"] = file
                yield doc_dict

In [14]:
def preprocessed_text_in_df(df):
    df['preprocessed_text'] = ''
    df['lemmatized_text'] = ''
    df['lemmatized_text_pos_tags'] = ''

    for i in tqdm_notebook(range(len(df))):
        preprocessed_text = process_text(df.iloc[i]['text'])
        df['preprocessed_text'].iloc[i] = preprocessed_text['preprocessed_text']
        df['lemmatized_text'].iloc[i] = preprocessed_text['lemmatized_text']
        df['lemmatized_text_pos_tags'].iloc[i] = preprocessed_text['lemmatized_text_pos_tags']

In [16]:
preprocessed_text_in_df(df_vacancies_4k)

C:\ProgramData\Anaconda3\envs\all_my_projects_3_6\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [17]:
df_vacancies_4k.head()

,id,title,text,labels,preprocessed_text,lemmatized_text,lemmatized_text_pos_tags
0,20001586,Инженер-проектировщик решений по передаче данных,Инженер-проектировщик решений по передаче данн...,20,"[инженер, проектировщик, решений, по, передаче...","[инженер, проектировщик, решение, передача, да...","[инженер_NOUN, проектировщик_NOUN, решение_NOU..."
1,20001746,Архитектор решений,Архитектор решений Обязанности: разработка ар...,17,"[архитектор, решений, обязанности, разработка,...","[архитектор, решение, обязанность, разработка,...","[архитектор_NOUN, решение_NOUN, обязанность_NO..."
2,20001909,Python-разработчик в Район,Python-разработчик в Район Новый сервис Яндекс...,14,"[python, разработчик, в, район, новый, сервис,...","[python, разработчик, район, новый, сервис, ян...","[python_, разработчик_NOUN, район_NOUN, новый_..."
3,20002793,Системный администратор/Программист Delphi,Системный администратор/Программист Delphi Обя...,"14,10","[системный, администратор, программист, delphi...","[системный, администратор, программист, delphi...","[системный_ADJF, администратор_NOUN, программи..."
4,20004542,IT аналитик (электронные торговые площадки),IT аналитик (электронные торговые площадки) ...,8,"[it, аналитик, электронные, торговые, площадки...","[it, аналитик, электронный, торговый, площадка...","[it_, аналитик_NOUN, электронный_ADJF, торговы..."


In [18]:
preprocessed_text_in_df(df_vacancies_22k)

C:\ProgramData\Anaconda3\envs\all_my_projects_3_6\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
df_vacancies_22k.head()

,id,is_prog,is_test,label_true,title,text,preprocessed_text,lemmatized_text,lemmatized_text_pos_tags
0,24992389,False,True,101,Специалист офиса продаж и обслуживания,Специалист офиса продаж и обслуживания Ты наш ...,"[специалист, офиса, продаж, и, обслуживания, т...","[специалист, офис, продажа, обслуживание, наш,...","[специалист_NOUN, офис_NOUN, продажа_NOUN, обс..."
1,25195139,False,True,101,Специалист салона связи Beeline,Специалист салона связи Beeline Ты наш человек...,"[специалист, салона, связи, beeline, ты, наш, ...","[специалист, салон, связь, beeline, наш, челов...","[специалист_NOUN, салон_NOUN, связь_NOUN, beel..."
2,22995930,False,True,101,Специалист отдела продаж Билайн,Специалист отдела продаж Билайн Ты наш человек...,"[специалист, отдела, продаж, билайн, ты, наш, ...","[специалист, отдел, продажа, билайн, наш, чело...","[специалист_NOUN, отдел_NOUN, продажа_NOUN, би..."
3,23754493,False,True,101,Продавец-консультант,"Продавец-консультант Ты наш человек, если поня...","[продавец, консультант, ты, наш, человек, если...","[продавец, консультант, наш, человек, понятие,...","[продавец_NOUN, консультант_NOUN, наш_ADJF, че..."
4,23841958,False,True,101,Продавец консультант в салон связи Билайн,Продавец консультант в салон связи Билайн Ты н...,"[продавец, консультант, в, салон, связи, билай...","[продавец, консультант, салон, связь, билайн, ...","[продавец_NOUN, консультант_NOUN, салон_NOUN, ..."


In [5]:
import pickle
def read_from_pickle(filename):
    with open(filename, 'rb') as f:
        data_new = pickle.load(f)
    return data_new

def write_to_pickle(filename, data):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [23]:
filename = f'{dataset_dir}\\df_vacancies_4k_preprocessed_text.pickle'
write_to_pickle(filename, df_vacancies_4k)

In [22]:
filename = f'{dataset_dir}\\df_vacancies_22k_preprocessed_text.pickle'
write_to_pickle(filename, df_vacancies_22k)

In [22]:
filename = f'{dataset_dir}\\df_vacancies_22k_preprocessed_text.pickle'
df_vacancies_22k = read_from_pickle(filename)

In [23]:
filename = f'{dataset_dir}\\df_vacancies_4k_preprocessed_text.pickle'
df_vacancies_4k = read_from_pickle(filename)

In [24]:
df_vacancies_4k.head()

,id,title,text,labels,preprocessed_text,lemmatized_text,lemmatized_text_pos_tags
0,20001586,Инженер-проектировщик решений по передаче данных,Инженер-проектировщик решений по передаче данн...,20,"[инженер, проектировщик, решений, по, передаче...","[инженер, проектировщик, решение, передача, да...","[инженер_NOUN, проектировщик_NOUN, решение_NOU..."
1,20001746,Архитектор решений,Архитектор решений Обязанности: разработка ар...,17,"[архитектор, решений, обязанности, разработка,...","[архитектор, решение, обязанность, разработка,...","[архитектор_NOUN, решение_NOUN, обязанность_NO..."
2,20001909,Python-разработчик в Район,Python-разработчик в Район Новый сервис Яндекс...,14,"[python, разработчик, в, район, новый, сервис,...","[python, разработчик, район, новый, сервис, ян...","[python_, разработчик_NOUN, район_NOUN, новый_..."
3,20002793,Системный администратор/Программист Delphi,Системный администратор/Программист Delphi Обя...,"14,10","[системный, администратор, программист, delphi...","[системный, администратор, программист, delphi...","[системный_ADJF, администратор_NOUN, программи..."
4,20004542,IT аналитик (электронные торговые площадки),IT аналитик (электронные торговые площадки) ...,8,"[it, аналитик, электронные, торговые, площадки...","[it, аналитик, электронный, торговый, площадка...","[it_, аналитик_NOUN, электронный_ADJF, торговы..."


In [25]:
df_vacancies_22k.head()

,id,is_prog,is_test,label_true,title,text,preprocessed_text,lemmatized_text,lemmatized_text_pos_tags
0,24992389,False,True,101,Специалист офиса продаж и обслуживания,Специалист офиса продаж и обслуживания Ты наш ...,"[специалист, офиса, продаж, и, обслуживания, т...","[специалист, офис, продажа, обслуживание, наш,...","[специалист_NOUN, офис_NOUN, продажа_NOUN, обс..."
1,25195139,False,True,101,Специалист салона связи Beeline,Специалист салона связи Beeline Ты наш человек...,"[специалист, салона, связи, beeline, ты, наш, ...","[специалист, салон, связь, beeline, наш, челов...","[специалист_NOUN, салон_NOUN, связь_NOUN, beel..."
2,22995930,False,True,101,Специалист отдела продаж Билайн,Специалист отдела продаж Билайн Ты наш человек...,"[специалист, отдела, продаж, билайн, ты, наш, ...","[специалист, отдел, продажа, билайн, наш, чело...","[специалист_NOUN, отдел_NOUN, продажа_NOUN, би..."
3,23754493,False,True,101,Продавец-консультант,"Продавец-консультант Ты наш человек, если поня...","[продавец, консультант, ты, наш, человек, если...","[продавец, консультант, наш, человек, понятие,...","[продавец_NOUN, консультант_NOUN, наш_ADJF, че..."
4,23841958,False,True,101,Продавец консультант в салон связи Билайн,Продавец консультант в салон связи Билайн Ты н...,"[продавец, консультант, в, салон, связи, билай...","[продавец, консультант, салон, связь, билайн, ...","[продавец_NOUN, консультант_NOUN, салон_NOUN, ..."


In [20]:
print(len(df_vacancies_22k))
print(len(df_vacancies_4k))

22810
4290


In [76]:
def filter_nums(tokens):
    return [token for token in tokens if(token.isdigit()==False)]

df_vacancies_22k['lemmatized_text_wo_num'] = df_vacancies_22k['lemmatized_text_wo_num'].apply(filter_nums)
df_vacancies_4k['lemmatized_text_wo_num'] = df_vacancies_4k['lemmatized_text_wo_num'].apply(filter_nums)

In [98]:
# tokens = []
# for line in df_vacancies_4k['lemmatized_text_wo_num']:
#     for token in line:
#         tokens.append(token)

# print(len(tokens))
# for token in tokens:
#     if token.find('000') == True:
#         print(token)

In [99]:
# from sklearn.feature_extraction.text import CountVectorizer
# import nltk

# max_features = None
# max_df = 1.0
# internal_stopwords = []



# df = pd.DataFrame.from_dict(data=dicts_texts_vacancies_4k, orient='index')

# documents_corpus = df[0].values

# vectorizer = CountVectorizer(max_features=max_features,
#                                          max_df=max_df,
#                                          stop_words=internal_stopwords, tokenizer = nltk.tokenize.word_tokenize).fit(documents_corpus)


In [100]:
# vectorizer

# Собираем словарь id:text

In [58]:
import logging
def get_dict_id_texts_from_df_tokens(df, col_id, col_text):
    dict_texts = {}
    for i in tqdm_notebook(range(len(df))):
        #new_id = str(df.iloc[i]['id'])
        new_id = df.iloc[i]['id']
        dict_texts[new_id] = ' '.join(df.iloc[i][col_text])
    logging.info('Длина словаря: ' + str(len(dict_texts)))
    return dict_texts

In [59]:
dicts_texts_vacancies_4k = get_dict_id_texts_from_df_tokens(df_vacancies_4k, 'id', 'lemmatized_text_wo_num')
print(len(dicts_texts_vacancies_4k))

4290


In [60]:
dicts_texts_vacancies_22k = get_dict_id_texts_from_df_tokens(df_vacancies_22k,'id', 'lemmatized_text_wo_num')
print(len(dicts_texts_vacancies_22k))

22810


# Строим данные в формате UCI

In [93]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

class BagOfWordsModel(object):
    OUT_FOLDER = 'out\\'

    def __init__(self, id_document_dict, max_features=None, max_df=1.0, stopwords_addon = []):
        """Builds bow model.
        Args:
            id_document_dict: ids of documents and theirs contents in format
                "{id: 'text', ...}"
            max_features: If not None, build a vocabulary that only consider the top
                max_features ordered by term frequency across the corpus.
                This parameter is ignored if vocabulary is not None.
            max_df: When building the vocabulary ignore terms that have a
                document frequency strictly higher than the given threshold
                (corpus-specific stop words). If float, the parameter
                represents a proportion of documents, integer absolute counts.
                This parameter is ignored if vocabulary is not None.
        """
        self.logger = logging.getLogger(__name__)
        self.logger.info("Building bag-of-words model with max_features={0}, max_df={1}".format(max_features, max_df))
        self.logger.info("Size of data set: " + str(len(id_document_dict)))

        if len(id_document_dict) != 0:
            self.logger.info("Building pandas dataframe")
            df = pd.DataFrame.from_dict(data=id_document_dict, orient='index')
            self.logger.info("Built pandas dataframe")
            ids = df.index
            self.index2id = dict(enumerate(ids))
            self.id2index = {v: k for k, v in self.index2id.items()}
            documents_corpus = df[0].values  # 1-dim np.array.
            # documents_corpus = documents_corpus.astype(unicode)
            del df
            if max_features is None:
                self.logger.info(
                    "Training CountVectorizer with all {0} features".format(len(ids)))
            else:
                self.logger.info(
                    "Training CountVectorizer with max {0} features".format(max_features))
            
            internal_stopwords = []
            internal_stopwords.extend(stopwords.words('russian'))
            internal_stopwords.extend(stopwords.words('english'))
            internal_stopwords.extend(stopwords_addon)
            
            vectorizer = CountVectorizer(max_features=max_features,
                                         max_df=max_df,
                                         stop_words=internal_stopwords,
                                         tokenizer = nltk.tokenize.word_tokenize).fit(documents_corpus)
            
            self.logger.info("Trained vectorizer with {0} features".format(len(vectorizer.get_feature_names())))
            self.logger.info("Building bag-of-words model")
            bow = vectorizer.transform(documents_corpus)
            self.logger.info("Done")

            self.url_ids = ids
            self.bow_sparse_matrix = bow
            self.feature_names = vectorizer.get_feature_names()  # mapping from url_id to url
            self.vocabulary = vectorizer.vocabulary_  # mapping from url to url_id
            self.shape = self.bow_sparse_matrix.shape

    def get_index(self, doc_id):
        return self.id2index[doc_id]

    def get_doc_id(self, index):
        return self.index2id[index]

    def get_feature_id(self, feature_name):
        return self.vocabulary.get(feature_name)

    def get_feature_name(self, feature_id):
        return self.feature_names[feature_id]

    def toarray(self):
        return self.bow_sparse_matrix.toarray()

    def to_uci(self, model_name='bow', save_folder=OUT_FOLDER):
        import os.path
        import codecs
        if self.bow_sparse_matrix is None:
            self.logger.error("Model is None.")
            return
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        filenames = model_name
        docword_name = os.path.join(save_folder, 'docword.' + filenames + '.txt')
        vocab_name = os.path.join(save_folder, 'vocab.' + filenames + '.txt')
        with codecs.open(docword_name, 'w', encoding='utf-8') as docword_f, \
             codecs.open(vocab_name, 'w', encoding='utf-8') as vocab_f:
            
            urls_count = self.shape[0]
            words_count = self.shape[1]
            # Fill vocab_f file
            self.logger.info("Start filling {0}".format(vocab_name))
            for i in tqdm_notebook(range(words_count)):
#                 if(self.get_feature_name(i).isdigit()):
#                     print(self.get_feature_name(i))
                vocab_f.write(self.get_feature_name(i) + '\n')
            self.logger.info("Done.")
            # Fill docword_f file
            self.logger.info("Start filling {0}".format(docword_name))
            docword_f.write(str(urls_count) + '\n')
            docword_f.write(str(words_count) + '\n')
            docword_f.write(str(self.bow_sparse_matrix.nnz) + '\n')
            # nnz_position = docword_f.tell() # We fill this line later with nnz_counter.
            # nnz_counter = 0 # The number of nonzero counts in the bag-of-words.
            nnz_x, nnz_y = self.bow_sparse_matrix.nonzero()
            for x, y in zip(nnz_x, nnz_y):
                # nnz_counter += len(url_sparse_vector)
                docword_f.write(str(x + 1) + ' ' + str(y + 1) + ' ' + str(
                    self.bow_sparse_matrix[x, y]) + '\n')
            self.logger.info("Done.")

In [94]:
bow_model_4k = BagOfWordsModel(dicts_texts_vacancies_4k, stopwords_addon=[])

In [95]:
bow_model_4k.to_uci(save_folder=f'{dataset_dir}\\uci_4k\\')

In [96]:
bow_model_22k = BagOfWordsModel(dicts_texts_vacancies_22k, stopwords_addon=[])

In [97]:
bow_model_22k.to_uci(save_folder=f'{dataset_dir}\\uci_22k\\')

# Строим данные в формате vowpal_wabbit

In [115]:
import nltk
from nltk.corpus import stopwords
morph = pymorphy2.MorphAnalyzer()

def make_vowpal_wabbit_str_from_dict_texts(dict_texts, morph, stopwords_addon=[], case = "ordered"):
    s = ''
    keys = list(dict_texts.keys())
    for i in tqdm_notebook(range(len(keys))):
        text = dict_texts[keys[i]]
    
        word=nltk.word_tokenize(text)    
        word_ws=[w.lower() for w in word if w.isalpha() ] #исключение слов и символов      
        
        
        internal_stopwords = []
        internal_stopwords.extend(stopwords.words('russian'))
        internal_stopwords.extend(stopwords.words('english'))
        internal_stopwords.extend(stopwords_addon)
        
        word_w=[w for w in word_ws if w not in internal_stopwords ] #нижний регистр  
    
        
        lem = [morph.parse(word)[0].normal_form for word in word_w]
    
        lema=[w for w in lem if w.isalpha() and len(w)>1]

        if(case == "ordered"):
            s = s + "|text" +" "+(" ").join(lema)+'\n'
        if(case == "frequency"):
            #z = (" ").join(lema)
            freq=nltk.FreqDist(lema) # распределение слов в i -м документе по частоте
            z=[] # обновление списка для нового документа

            z=[(key+":"+str(val)) for key,val in freq.items() if val>1] # частота упоминания через : от слова

            s = s + "|text" +" "+(" ").join(z)+'\n'       
    return s

In [116]:
# dir(dicts_texts_vacancies_4k)
# dicts_texts_vacancies_4k

In [117]:
cases = ["ordered", "frequency"]
dicts = {"vacancies_4k":dicts_texts_vacancies_4k, "vacancies_22k":dicts_texts_vacancies_22k}

for key in dicts:
    for case in cases:
        print(key, case)
        vw = make_vowpal_wabbit_str_from_dict_texts(dicts[key], morph, case = case)

        filename_wv = f'{dataset_dir}\\vw\\vw_{key}_{case}.txt'
        f=open(filename_wv,'w', encoding='utf')
        f.write(vw)        
        f.close()

vacancies_4k ordered


vacancies_4k frequency


vacancies_22k ordered


vacancies_22k frequency


# Добавляем тематические вектора из разных моделей ARTM

In [26]:
experiment = f"vacancies_08_22k_300_500_60"
experiment_dir = f"C:\\Users\\Ivan\\PycharmProjects\\all_my_projects_3_6\\bigartm vacancies\\results\\{experiment}"

In [27]:
import os

artm_theha_matrix_dir = f"{experiment_dir}\\matrixes\\"

def generator_files_in_dir(top_directory, extension=''):
    for root, dirs, files in os.walk(top_directory):
        for filename in filter(lambda fname: fname.endswith(extension), files):
            filename_full = os.path.join(root, filename)
            if os.path.isfile(filename_full) is True:
                yield filename_full
    pass
         
_filenames = [filename for filename in generator_files_in_dir(artm_theha_matrix_dir) if filename.find('theta')!=-1]

_models = [filename.split('\\')[-1].split('.csv')[0][:-6] for filename in _filenames]
print(_models)

['300_0.0_0.0_0_60', '500_0.0_0.0_0_60']


# Выбираем модели

In [28]:
models = ['300_0.0_0.0_0_60', '500_0.0_0.0_0_60']
filenames = [filename for filename in _filenames for model in models if model in filename]
print(filenames)

['C:\\Users\\Ivan\\PycharmProjects\\all_my_projects_3_6\\bigartm vacancies\\results\\vacancies_08_22k_300_500_60\\matrixes\\300_0.0_0.0_0_60_theta.csv', 'C:\\Users\\Ivan\\PycharmProjects\\all_my_projects_3_6\\bigartm vacancies\\results\\vacancies_08_22k_300_500_60\\matrixes\\500_0.0_0.0_0_60_theta.csv']


In [29]:
df_work = df_vacancies_22k

In [30]:
for filename in filenames:
    print(f"Добавляем вектора из файла {filename}")
    name_model = filename.split('\\')[-1].split('.csv')[0]
    
    df_theta = pandas.read_csv(filename, sep=",", encoding='utf')
    
    df_theta.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
    
    del df_theta['id']
    df_theta = df_theta.T
    print(len(df_theta))
    
    df_theta = df_theta.reset_index()
    df_theta['index'] = df_theta['index'].astype('int64')
    
    df_theta = df_theta.sort_values(['index'])
    df_theta.head()
    

    del df_theta['index']
    df_theta.head()

    lst_row = df_theta.apply(lambda x: x.tolist(), axis=1)
    se = pandas.Series(lst_row)
    df_work[f'{name_model}'] = se.values

df_work.head(2)

Добавляем вектора из файла C:\Users\Ivan\PycharmProjects\all_my_projects_3_6\bigartm vacancies\results\vacancies_08_22k_300_500_60\matrixes\300_0.0_0.0_0_60_theta.csv
22810
Добавляем вектора из файла C:\Users\Ivan\PycharmProjects\all_my_projects_3_6\bigartm vacancies\results\vacancies_08_22k_300_500_60\matrixes\500_0.0_0.0_0_60_theta.csv
22810


,id,is_prog,is_test,label_true,title,text,preprocessed_text,lemmatized_text,lemmatized_text_pos_tags,300_0.0_0.0_0_60_theta,500_0.0_0.0_0_60_theta
0,24992389,False,True,101,Специалист офиса продаж и обслуживания,Специалист офиса продаж и обслуживания Ты наш ...,"[специалист, офиса, продаж, и, обслуживания, т...","[специалист, офис, продажа, обслуживание, наш,...","[специалист_NOUN, офис_NOUN, продажа_NOUN, обс...","[9.985508e-05, 1.0782002e-06, 2.3501414e-05, 0...","[0.00056593685, 2.3404924e-08, 6.9312387e-07, ..."
1,25195139,False,True,101,Специалист салона связи Beeline,Специалист салона связи Beeline Ты наш человек...,"[специалист, салона, связи, beeline, ты, наш, ...","[специалист, салон, связь, beeline, наш, челов...","[специалист_NOUN, салон_NOUN, связь_NOUN, beel...","[9.664870000000001e-05, 1.0274571e-06, 2.27085...","[0.0005477639, 2.0375256999999998e-08, 6.50756..."


In [21]:
#del df_work['180_0_0_0_theta']

In [31]:
df_work.to_csv('d:\\vacancies_08_22k_300_500_60_results.csv')

# Добавляем ключевые слова из моделей в dataframe из разных моделей ARTM

In [30]:
import os

artm_theha_matrix_dir = f"{experiment_dir}\\top_tokens\\"

def generator_files_in_dir(top_directory, extension=''):
    for root, dirs, files in os.walk(top_directory):
        for filename in filter(lambda fname: fname.endswith(extension), files):
            filename_full = os.path.join(root, filename)
            if os.path.isfile(filename_full) is True:
                yield filename_full
    pass
         
_filenames_top_tokens = [filename for filename in generator_files_in_dir(artm_theha_matrix_dir) if filename.find('.csv')!=-1]
print(_filenames_top_tokens)
# _models = [filename.split('\\')[-1].split('.csv')[0] for filename in _filenames]
# print(_models)

['C:\\Users\\Ivan\\PycharmProjects\\all_my_projects_3_6\\bigartm vacancies\\results\\vacancies_04_optimal_num_topics_for_22k 60 (100-200)\\top_tokens\\100_0_0_0.csv', 'C:\\Users\\Ivan\\PycharmProjects\\all_my_projects_3_6\\bigartm vacancies\\results\\vacancies_04_optimal_num_topics_for_22k 60 (100-200)\\top_tokens\\120_0_0_0.csv', 'C:\\Users\\Ivan\\PycharmProjects\\all_my_projects_3_6\\bigartm vacancies\\results\\vacancies_04_optimal_num_topics_for_22k 60 (100-200)\\top_tokens\\140_0_0_0.csv', 'C:\\Users\\Ivan\\PycharmProjects\\all_my_projects_3_6\\bigartm vacancies\\results\\vacancies_04_optimal_num_topics_for_22k 60 (100-200)\\top_tokens\\160_0_0_0.csv', 'C:\\Users\\Ivan\\PycharmProjects\\all_my_projects_3_6\\bigartm vacancies\\results\\vacancies_04_optimal_num_topics_for_22k 60 (100-200)\\top_tokens\\180_0_0_0.csv', 'C:\\Users\\Ivan\\PycharmProjects\\all_my_projects_3_6\\bigartm vacancies\\results\\vacancies_04_optimal_num_topics_for_22k 60 (100-200)\\top_tokens\\200_0_0_0.csv']


In [34]:
# for filename in _filenames_top_tokens[:1]:
print(f"Добавляем ключевые слова из файла {filename}")
name_model = filename.split('\\')[-1].split('.csv')[0]

df_top_tokens = pandas.read_csv(filename, sep=",", encoding='utf')

df_top_tokens.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
df_top_tokens.head()
del df_theta['id']
#     df_theta = df_theta.T
#     print(len(df_theta))
    
#     df_theta = df_theta.reset_index()
#     df_theta['index'] = df_theta['index'].astype('int64')
    
#     df_theta = df_theta.sort_values(['index'])
#     df_theta.head()
    

#     del df_theta['index']
#     df_theta.head()

#     lst_row = df_theta.apply(lambda x: x.tolist(), axis=1)
#     se = pandas.Series(lst_row)
#     df_work[f'{name_model}'] = se.values

#df_work.head(2)

Добавляем ключевые слова из файла C:\Users\Ivan\PycharmProjects\all_my_projects_3_6\bigartm vacancies\results\vacancies_04_optimal_num_topics_for_22k 60 (100-200)\top_tokens\100_0_0_0.csv


,id,0,1,2,3,4,5,6,7,8,9,10
0,0,topic_0,технический,работа,инженер,профильный,образование,сервисный,специальность,стаж,условие,высокий
1,1,topic_1,коллектив,дружный,молодая,комфортный,возможность,труд,интересный,молодой,оплата,позитивный
2,2,topic_2,день,отпуск,оплачивать,больничный,занятость,полный,часы,обед,время,свободный
3,3,topic_3,бизнес,процесс,анализ,сбор,требование,внутренний,потребность,выявление,постановка,заказчик
4,4,topic_4,задача,интересный,сложный,поставить,решать,реализовать,подход,нестандартный,выполнять,требовать
